# The smart data analysis assistant 

#### File system and data handling

In [111]:
import pathlib as path

In [114]:
data = path.Path("datasets")
if not data.exists:
    data.mkdir(parents=True, exist_ok=True)
else :
    print("Path {} already exists".format(data))

Path datasets already exists


##### Validate input. Returns boolean

In [5]:


# read in excel file
def validate_file(source):
    if path.Path(source).is_file():
        if path.Path(source).suffix in [".csv",".xlsx"]:
            print("file is valid")
            return True
        else:
            print("unaccepted file format")
            print(path.Path(source).suffix)
            return False
    else:
        print("file is invalid")
        return False

##### read input. Returns tuple (dataframe/None,source)

In [115]:
#read dataset if valid
import pandas as pd
def read_input(source):
    is_valid = validate_file(source)
    if is_valid:
        if path.Path(source).suffix == ".csv":
            data = pd.read_csv(source)
            return (data,source)

        elif path.Path(source).suffix == ".xlsx":
            data = pd.read_excel(source)
            return (data,source)
    return (None,source)

##### Test read_input function

In [116]:
#read invalid file
res = read_input("")
if res[0] is None:
    print("Provide file ' {} ' in ivalid".format(res[1]))


file is invalid
Provide file '  ' in ivalid


In [117]:
#read valid file
res = read_input("datasets/diabetes-data.csv")
print("file '{}' is valid".format(res[1]))

file is valid
file 'datasets/diabetes-data.csv' is valid


##### Grab dataframe as first item in tuple

In [16]:
data = res[0]

##### Trying to describe data

In [17]:
# Take a peep at the data
data.head()

,AGE,Gender,FPG,HbA1c,Noctouria,Polyuria,Weight_loss,Vomiting,Nausea,Polydipsia,Polyphagia,Headache,BMI,Outcome
0,52,F,7.0,6.7,True,True,True,False,False,False,False,False,31.0,positive
1,57,M,10.8,8.0,False,False,False,False,False,False,False,False,32.0,positive
2,56,F,3.6,5.0,False,False,False,False,False,False,False,False,23.0,negative
3,32,F,7.5,6.8,False,False,False,False,False,False,False,False,31.0,positive
4,51,M,5.7,6.2,False,False,False,False,False,False,False,False,20.0,negative


In [21]:
#Get shape of data
data.shape

(3415, 14)

In [20]:
#Get columns
data.columns

#

Index(['AGE', 'Gender', 'FPG', 'HbA1c', 'Noctouria', 'Polyuria', 'Weight_loss',
       'Vomiting', 'Nausea', 'Polydipsia', 'Polyphagia', 'Headache', 'BMI',
       'Outcome'],
      dtype='object')

In [28]:
#Get data types
data.dtypes

AGE              int64
Gender          object
FPG            float64
HbA1c          float64
Noctouria         bool
Polyuria          bool
Weight_loss       bool
Vomiting          bool
Nausea            bool
Polydipsia        bool
Polyphagia        bool
Headache          bool
BMI            float64
Outcome         object
dtype: object

In [23]:
# Get numeric fields
numeric_data = data.select_dtypes(exclude=['O','bool'])
numeric_data.head()

,AGE,FPG,HbA1c,BMI
0,52,7.0,6.7,31.0
1,57,10.8,8.0,32.0
2,56,3.6,5.0,23.0
3,32,7.5,6.8,31.0
4,51,5.7,6.2,20.0


In [30]:
# Describe numeric data
numeric_data.describe()

,AGE,FPG,HbA1c,BMI
count,3415.000000,3415.000000,3415.000000,3408.000000
mean,50.310688,7.314310,6.980700,29.939830
std,10.772496,2.084517,1.576805,5.082226
min,5.000000,1.800000,2.300000,0.000000
25%,44.000000,5.700000,5.900000,25.000000
50%,52.000000,7.600000,6.900000,31.000000
75%,56.000000,8.500000,7.900000,34.000000
max,209.000000,17.600000,14.000000,47.000000


##### Function to get null value statistics from dataset

In [120]:
#What fields have  null values

def get_null_stats(data):
    
    #What is the percentage of these null values 
    null_fields = get_null_fields(data)[0]
    null_fields_names = get_null_fields(data)[1]

    for col in null_fields_names:
        null_fields[col] = null_fields[col]*100/len(data[col])

    return null_fields


def get_null_fields(data):
    null_stats = data.isnull().sum()
    null_fields = null_stats[null_stats>0]
    null_fields
    #What is the percentage of these null values 
    null_fields_names = null_fields.index

    return (null_fields,null_fields_names)


##### Get summary and next step suggestions on any null values

In [121]:

def print_summary(null_stats):
    nulls_exist = len(null_stats) > 0
    if nulls_exist:
        print("dataset has null numeric values")
        print("........")
        print("summary")
        print("...................")
        print(null_stats)
        print("...................")
        print("...................")
        print("""Possible actions to take :
            1. Drop rows with any missing values
            2. Drop columns with miissing values
            3. Inpute missing data
            """)
    else:
        print("dataset has no null values")

##### Use functions to get null statistics

In [105]:
# Get null stats
numeric_data_null_stats = get_null_stats(numeric_data)

# Print summary based on null stats
print_summary(numeric_data_null_stats)

dataset has null numeric values
........
summary
...................
BMI    0.204978
dtype: float64
...................
...................
Possible actions to take :
            1. Drop rows with any missing values
            2. Drop columns with miissing values
            3. Inpute missing data
            


/var/folders/1g/f8q40hm92l30jzph0wdckn8h0000gn/T/ipykernel_21483/1344933681.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.20497803806734993' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  null_fields[col] = null_fields[col]*100/len(data[col])


##### Define null remover

In [122]:
def imputer(columns):
    print('impute missing data in colums {}'.format(columns))

In [123]:
def null_remover(decision,df,columns=[]):
    if decision == "drop_rows":
       cleaned = df.dropna(axis=0)
       
       print("Dropped {} rows from dataset".format(len(df)-len(cleaned)))
       print("...................")

    elif decision == "drop_cols":
       if len(columns) == 0:
           print("Please provide a nonempty column list")
           print("...................")

       cleaned = df.drop(columns,axis=1)
       print("Dropped columns {} from dataset".format(columns))
       print("...................")


    elif decision == "inpute":
        imputer(columns)
    return cleaned


##### Use null remover

In [124]:
# now lets deal with null values for numeric data
cleaned = null_remover("drop_cols",numeric_data,get_null_fields(numeric_data)[1])
print(cleaned.isnull().any())
cleaned.shape


Dropped columns Index(['BMI'], dtype='object') from dataset
...................
AGE      False
FPG      False
HbA1c    False
dtype: bool


(3415, 3)

In [125]:
# now lets deal with null values
cleaned = null_remover("drop_rows",numeric_data)
print(cleaned.isnull().any())
cleaned.shape


Dropped 7 rows from dataset
...................
AGE      False
FPG      False
HbA1c    False
BMI      False
dtype: bool


(3408, 4)

In [81]:
categorical_data = data.select_dtypes(exclude=['int64','float64'])
categorical_data.head()

,Gender,Noctouria,Polyuria,Weight_loss,Vomiting,Nausea,Polydipsia,Polyphagia,Headache,Outcome
0,F,True,True,True,False,False,False,False,False,positive
1,M,False,False,False,False,False,False,False,False,positive
2,F,False,False,False,False,False,False,False,False,negative
3,F,False,False,False,False,False,False,False,False,positive
4,M,False,False,False,False,False,False,False,False,negative


# Describe categorical data

In [95]:
categorical_data.describe()

,Gender,Noctouria,Polyuria,Weight_loss,Vomiting,Nausea,Polydipsia,Polyphagia,Headache,Outcome
count,3414,3415,3415,3415,3415,3415,3415,3415,3415,3415
unique,2,2,2,2,2,2,2,2,2,2
top,M,True,True,True,False,False,False,False,False,positive
freq,1822,1846,1846,1846,3413,3413,3414,3414,3413,2239


In [109]:
# Get null stats for categorical data
categorical_data_null_stats = get_null_stats(categorical_data)

print_summary(categorical_data_null_stats)


dataset has null numeric values
........
summary
...................
Gender    0.029283
dtype: float64
...................
...................
Possible actions to take :
            1. Drop rows with any missing values
            2. Drop columns with miissing values
            3. Inpute missing data
            


/var/folders/1g/f8q40hm92l30jzph0wdckn8h0000gn/T/ipykernel_21483/1344933681.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.029282576866764276' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  null_fields[col] = null_fields[col]*100/len(data[col])


In [110]:
# Get null stats for whole dataset

data_null_stats = get_null_stats(data)

print_summary(data_null_stats)

dataset has null numeric values
........
summary
...................
Gender    0.029283
BMI       0.204978
dtype: float64
...................
...................
Possible actions to take :
            1. Drop rows with any missing values
            2. Drop columns with miissing values
            3. Inpute missing data
            


/var/folders/1g/f8q40hm92l30jzph0wdckn8h0000gn/T/ipykernel_21483/1344933681.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.029282576866764276' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  null_fields[col] = null_fields[col]*100/len(data[col])
